In [1]:
#installing psycopg2
!pip install psycopg2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 1.3 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.7-cp311-cp311-macosx_10_9_universal2.whl size=245062 sha256=b15d9433f19b01303ee7b2ebbcb4714c52a64c3e3ffe2349ab4dc7f929ccd88e
  Stored in directory: /Users/HananNawaz/Library/Caches/pip/wheels/ca/a2/1f/9f85470e2065e7a4089baa6c47cfa57a2f8f8e78e07390c5b4
Successfully built psycopg2

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
#importing psycopg2
import psycopg2

In [3]:
#function to connect to database
def connect_database():
    try:   
        #connecting to default database
        conn = psycopg2.connect(
            dbname='postgres',
            user='postgres',
            password='',
            host='localhost'
        )
        
        #creating cursor
        cur = conn.cursor()
        
        #making auto commmit true
        conn.autocommit = True
        
        #creating new database
        cur.execute("CREATE DATABASE flight_crashes")
        
        #making autocommit false
        conn.autocommit = False
        
        #closing connection to database and cursor
        cur.close()
        conn.close()
        
        #connecting to new database we made in above section of code
        conn = psycopg2.connect(
                dbname='flight_crashes',
                user='postgres',
                password='',
                host='localhost'
        )
        
        #again creating cursor
        cur = conn.cursor()
        
        #returning cursor for later use
        return conn, cur

    except psycopg2.Error as e:
        
        #printing error if occur and returning None
        print('Error :' + str(e))
        return None;
        


In [8]:
#function for creating all tables
def create_tables(conn, cur):
    
    #crash date table query
    crash_date_table = "CREATE TABLE IF NOT EXISTS CrashDate (\
                        event_id SERIAL PRIMARY KEY, \
                        date varchar(20), \
                        time varchar(20))"
    
    #casualties table query
    casualties_table = "CREATE TABLE IF NOT EXISTS Casualties (\
                        casualties_id SERIAL PRIMARY KEY, \
                        casualties_plane INT, \
                        casualties_ground INT)"
    
    #crash summary table query
    crash_summary_table = "CREATE TABLE IF NOT EXISTS CrashSummary (\
                            summary_id SERIAL PRIMARY KEY, \
                            summary TEXT)"
    
    #operator table query
    operator_table = "CREATE TABLE IF NOT EXISTS Operator (\
                        operator_id SERIAL PRIMARY KEY, \
                        operator_name TEXT)"
    
    #plane type table query
    plane_type_table = "CREATE TABLE IF NOT EXISTS PlaneType (\
                        plane_id SERIAL PRIMARY KEY, \
                        plane_name TEXT, \
                        plane_registration TEXT, \
                        plane_cn_in TEXT)"
    
    #flight details table query
    flight_details_table = "CREATE TABLE IF NOT EXISTS FlightDetails (\
                            flight_id SERIAL PRIMARY KEY, \
                            flight_no TEXT, \
                            route TEXT, \
                            aboard_people bigint)"
    
    #crash table query
    crash_table = "CREATE TABLE IF NOT EXISTS Crash (\
                   crash_id SERIAL PRIMARY KEY, \
                   event_id bigint, \
                   operator_id bigint, \
                   plane_id bigint, \
                   flight_id bigint, \
                   location varchar(200), \
                   casualties_id bigint, \
                   summary_id bigint, \
                   FOREIGN KEY (event_id) REFERENCES CrashDate (event_id), \
                   FOREIGN KEY (operator_id) REFERENCES Operator (operator_id), \
                   FOREIGN KEY (plane_id) REFERENCES PlaneType (plane_id), \
                   FOREIGN KEY (flight_id) REFERENCES FlightDetails (flight_id), \
                   FOREIGN KEY (casualties_id) REFERENCES Casualties (casualties_id), \
                   FOREIGN KEY (summary_id) REFERENCES CrashSummary (summary_id))"
    
    #enabling autocommit
    conn.set_session(autocommit = True)
    
    try:
        #executing all tables query one by one
        cur.execute(crash_date_table)
        cur.execute(casualties_table)
        cur.execute(crash_summary_table)
        cur.execute(operator_table)
        cur.execute(plane_type_table)
        cur.execute(flight_details_table)
        cur.execute(crash_table)
        
    except psycopg2.Error as e:
        #printing error if occur and returning None
        print('Error:' + str(e))
    

In [9]:
#executing the connect_database() function in order to connect to db
connection_info = connect_database()

#checking if connection_info exists
if connection_info:
    conn, cur = connection_info
    
    #executing create_tables() function in order to create all tables
    create_tables(conn, cur)
else:
    print('error')